# Estimating Work from Home Model

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd
import numpy as np
from larch import P, X
import matplotlib.pyplot as plt

# Load data and prep model for estimation

In [2]:
# change the below path to your output folder
edb_output_dir = r'C:\ABM3_dev\outputs'
os.chdir(edb_output_dir)
modelname = "transponder_ownership"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [3]:
data.coefficients

,value,constrain
coefficient_name,,
coef_pct_multiple_auto,2.2250,F
coef_Expected_time_savings_up_to_03_min,6.8000,F
coef_Distance_above_two_miles,-0.0874,F
coef_Percent_detour_10_to_20,10.5140,F
coef_Transit_accessibility,-0.1150,F
coef_Transit_accessibility_is_zero,-0.3170,F
coef_Constant,-6.4760,F


#### Utility specification

In [4]:
data.spec

,Label,Description,Expression,No Pass,Pass
0,util_auto_ownership,auto_ownership zero would result in unavailabi...,@(df.auto_ownership==0) * (df.everyone_owns==0),0,-999
1,util_pct_multiple_auto,pct_multiple_auto,multiple_auto_hh_in_zone_share,0,coef_pct_multiple_auto
2,util_Expected_time_savings_up_to_03_min,"Expected time savings, up to 0.3 min","@np.minimum(df.avgtts,0.3)",0,coef_Expected_time_savings_up_to_03_min
3,util_Distance_above_two_miles,Distance above two miles,"@np.maximum(df.ml_dist-2,0)",0,coef_Distance_above_two_miles
4,util_Percent_detour_10_to_20,"Percent detour, 10% to 20%","@np.minimum(np.maximum(df.pctDetour-0.1,0),0.1)",0,coef_Percent_detour_10_to_20
5,util_Constant,Constant,1,0,coef_Constant


### Chooser data

In [5]:
data.chooser_data

,model_choice,override_choice,util_auto_ownership,util_pct_multiple_auto,util_Expected_time_savings_up_to_03_min,util_Distance_above_two_miles,util_Percent_detour_10_to_20,util_Constant,home_zone_id,income,...,everyone_owns,geoExp,qNonZero,pObs,Correction_for_nongeocoded_records,Correction_for_zeroauto_households,ml_dist,avgtts,pctDetour,override_choice_code
household_id,,,,,,,,,,,,,,,,,,,,,
1,False,False,0.0,0.0,0.30000,0.0000,0.0,1.0,13119,44996.25,...,0,1.06,0.837,0.009,0.058814,0.179701,0.4980,1.52527,0.09832,NaN
2,False,False,0.0,1.0,0.16124,5.2306,0.1,1.0,19303,38685.00,...,0,1.06,0.837,0.009,0.058814,0.179701,7.2306,0.16124,0.20751,NaN
3,False,False,0.0,0.0,0.23848,7.0316,0.0,1.0,12446,17175.00,...,0,1.06,0.837,0.009,0.058814,0.179701,9.0316,0.23848,0.00000,NaN
4,False,False,0.0,0.0,0.23571,1.6817,0.0,1.0,2699,37316.25,...,0,1.06,0.837,0.009,0.058814,0.179701,3.6817,0.23571,0.00000,NaN
5,False,False,0.0,0.0,0.22792,3.3084,0.0,1.0,13161,42760.00,...,0,1.06,0.837,0.009,0.058814,0.179701,5.3084,0.22792,0.00000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49758,False,False,0.0,0.0,0.22633,17.6129,0.0,1.0,5406,22370.00,...,0,1.06,0.837,0.009,0.058814,0.179701,19.6129,0.22633,0.00000,NaN
49759,False,False,0.0,0.0,0.23108,10.2910,0.0,1.0,11721,28805.00,...,0,1.06,0.837,0.009,0.058814,0.179701,12.2910,0.23108,0.00000,NaN
49760,False,False,0.0,0.0,0.22726,17.4927,0.0,1.0,7270,80659.00,...,0,1.06,0.837,0.009,0.058814,0.179701,19.4927,0.22726,0.00000,NaN


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [6]:
model.load_data()

req_data does not request avail_ca or avail_co but it is set and being provided


In [7]:
model.maximize_loglike(method="SLSQP")

┣                x: -999                                      -999.0000
┃                   0                                            0.0000
┃                   coef_Constant                               -6.4760
┃                   coef_Distance_above_two_miles               -0.0874
┃                   coef_Expected_time_savings_up_to_03_min      6.8000
┃                   coef_Percent_detour_10_to_20                10.5140
┃                   coef_pct_multiple_auto                       2.2250
┃                   dtype: float64
┣          loglike: 0.0
┣        d_loglike: -999                                       0.0
┃                   0                                          0.0
┃                   coef_Constant                              0.0
┃                   coef_Distance_above_two_miles              0.0
┃                   coef_Expected_time_savings_up_to_03_min    0.0
┃                   coef_Percent_detour_10_to_20               0.0
┃                   coef_pct_multiple_auto                     0.0
┃                   dtype: float64
┣              nit: 1
┣             nfev: 1
┣             njev: 1
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=2, microseconds=845212)
┣           method: 'SLSQP'
┣          n_cases: 49762
┣ iteration_number: 1
┣          logloss: -0.0

### Estimated coefficients

In [8]:
model.parameter_summary()

,Value,Null Value
-999,-999.,-999.00
0,0.00,0.00
coef_Constant,-6.48,0.00
coef_Distance_above_two_miles,-0.0874,0.00
coef_Expected_time_savings_up_to_03_min,6.80,0.00
coef_Percent_detour_10_to_20,10.5,0.00
coef_pct_multiple_auto,2.23,0.00


# Output Estimation Results

In [ ]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
)

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [ ]:
# can change result_dir to desired location
model.calculate_parameter_covariance()
model.to_xlsx(
    result_dir+"transponder_ownership.xlsx", 
    data_statistics=True,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [ ]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")